# ResNet Example

In [ ]:
!(pip show torch-summary || pip install torch-summary)

In [ ]:
!(test -d .git || test -d mp1 || git clone https://github.com/mike10004/csgy6953-mp1.git mp1)

In [ ]:
# change to correct branch here
# !(cd mp1 && git switch dev)

In [ ]:
!(cd mp1; git pull)

In [ ]:
import os
import sys
print("PYTHONPATH:", os.getenv("PYTHONPATH", None))
project_root_dir = '/content/mp1'
if not project_root_dir in sys.path:
    sys.path.append(project_root_dir)
print("\n".join(sys.path))

In [ ]:
import dlmp1
from pathlib import Path
print("imported", dlmp1, "from", Path(dlmp1.__file__).parent.parent)

In [ ]:
from dlmp1.models.resnet import ResNet18
import torchsummary

def summarize_model():
    model = ResNet18()
    stats = torchsummary.summary(model)
    print(stats)

summarize_model()

In [ ]:
import dlmp1.main
from dlmp1.main import TrainConfig

DO_TRAIN = False
CONFIG = TrainConfig(
    epoch_count=50,
    learning_rate=0.1,
    batch_size_train=200,
    verbose_scheduler=True,
)

if DO_TRAIN:
    
    dlmp1.main.perform(
        model_name="ResNet18",
        config=CONFIG,
    )
